# Generar Consultar Avanzadas

## Cruzar tablas: SQL JOIN

![](https://i.imgur.com/9c9jH95.jpg)

### Tipos de Joins

- JOIN(INNER)
- LEFT(OUTER)
- RIGHT(OUTER)
- FULLOUTER

Un join en PostgreSQL es una operación que combina filas de dos o más tablas en una sola tabla resultante. Se utiliza para recuperar información relacionada de diferentes tablas y para crear consultas más complejas.

Hay varios tipos de join en PostgreSQL, incluyendo:

- Inner join: solo incluye las filas que tienen una coincidencia en ambas tablas.

- Left join: incluye todas las filas de la tabla de la izquierda y las filas coincidentes de la tabla de la derecha. Si no hay una coincidencia, los valores en las columnas de la tabla de la derecha serán NULL.

- Right join: incluye todas las filas de la tabla de la derecha y las filas coincidentes de la tabla de la izquierda. Si no hay una coincidencia, los valores en las columnas de la tabla de la izquierda serán NULL.

- Full outer join: incluye todas las filas de ambas tablas, ya sea que haya una coincidencia o no. Si no hay una coincidencia, los valores en las columnas de una de las tablas serán NULL.

### Saber que pasajeros han tomado por lo menos un viaje:

    SELECT * FROM pasajero
    JOIN viaje ON (viaje.id_pasajero = pasajero.id);

y aqui contando los registros:

    SELECT COUNT(*) FROM pasajero
    JOIN viaje ON (viaje.id_pasajero = pasajero.id);

Encontramos 750 que coincide con el tamaño de la tabla *viaje*.

¡No lo se RICK, parece Falso!

### Que pasajeros no han tomado ni un solo viaje

    SELECT COUNT(*) FROM pasajero
    LEFT JOIN viaje ON (viaje.id_pasajero = pasajero.id)
    WHERE viaje.id IS NULL;

Nos da 24 


## Funciones Especiales Principales

Postgres tiene una lista de funciones especiales que ayudan a desarrollar tu aplicacion mucho mas rapido, si estan implementadas en otros motores de BD pueda que no funcionen igual:

- ON CONFLICT DO 

- RETURNING: muy util para no tener que hacer un SELECT despues de un INSERT.

- LIKE/ILIKE: realizar busquedas al estilo de expresiones regulares

- IS/ IS NOT: Comparar dos tipos de datos que no son estandar: saber si un campo es NULL o no lo es. 

### ON CONFLICT DO

supongamos que queremos insertar una nueva estacion con el ID=1, que ya existe, entonces habra un error: dulicate key.

Consultemos los 10 primieros registros:

    SELECT * FROM estacion LIMIT 10;

Ahora si el INSERT:

INSERT INTO public.estacion(
	id, nombre, direccion)
	VALUES (5, 'nombre', 'direccion')

Podemos usar ON CONFLICT de dos maneras: que no haga nada, y no haya error:

    INSERT INTO public.estacion(
		id, nombre, direccion)
		VALUES (5, 'nombre', 'direccion')
		ON CONFLICT DO NOTHING;

O que haga un update:

![](https://i.imgur.com/n9TnZFE.png)

Especificar la columna que entre en conflicto, en este caso *id*





### RETURNING

Supongamos queremos insertar una nueva estacion, y queremos verlo todo

![](https://i.imgur.com/UJbQ0Cs.png)

### LIKE y ILIKE

- LIKE: Case Sensitive
- ILIKE: case-insensitive.

LIKE y ILIKEtiene dos atributos:

- porcentaje(%): Uno o cualquier valor
- guion al piso(_): solamente uno

¿Que pasajeros empiezan por la letra *o*: 'o%':

    SELECT name
	FROM pasajero
	WHERE name LIKE 'o%'; 

No dara ningun resultado, probemos con ILIKE:

	SELECT COUNT(*)
	FROM pasajero
	WHERE name ILIKE 'o%'; 

Hay tres resultados ✅

#### Diferenciar entre % y _

Realizemos un nuevo script, para encontrar todos los que empiezan por la letra *a*:

	SELECT COUNT(*)
	FROM pasajero
	WHERE name ILIKE 'a%'; 

Hay 26 resultados, de los cuales tienen diferentes tamaños.

![](https://i.imgur.com/0XBGazc.png)

Ahora encontremos los que empiezan por la letra *a*, y solo de cuatro caracteres:

	SELECT name
	FROM pasajero
	WHERE name ILIKE 'a___'; 

Los resultados seran: Aila, Alon. 

### IS y IS NOT

Si queremos encontrar si un dato en particular es NULL o no lo es, no se puede simplemente con un =.

    SELECT COUNT(*)
	FROM tren
	WHERE modelo IS NOT NULL;

Arrojara 100 resultados.

## FUNCIONES ESPECIALES AVANZADAS

- COALESCE: comparar dos valores y determinar cual es nulo
- NULLIF
- GREATEST
- LEAST
- BLOQUES ANONIMOS: consultas avanzadas usando casi el mismo lenguaje de PHP o Python

Modificaremos un valor cualquiera en la tabla pasajero para que sea nulo, y una nueva consulta

![](https://i.imgur.com/dlWdta8.png)

Como ves el valor del nombre aparece en *null*. Si queremos que en vez de nulo aparezca otra cosa, por ejemplo, si no es Nulo que siga apareciendo el nombre, pero si lo es, que aparezca otra cosa:

![](https://i.imgur.com/Fq0oWfp.png)

### NULLIF

Si dos campos son iguales, retorna NULL:

    SELECT NULLIF (0,0);

De lo contrario retorna el valor del primer argumento, para este caso retornaria 12:

    SELECT NULLIF (12,-1);

### Eviatando el error de la division por cero con NULLIF

![](https://i.imgur.com/kOmxOqQ.png)

![](https://i.imgur.com/Li0QetK.png)

Con *NULLIF* al ser los dos valores iguales, entonces retorna NULL, pero lo interesante es que al dividir un numero entre NULL, el resultado es NULL, y no generara error. La palabra division, simplemente es un nombre cualquiera para la columna.

Como un ejemplo practico, considera el caso donde el *income* sea igual a *expenses*

![](https://i.imgur.com/vP7pxZt.png)

    SELECT investor_year,
        price_per_share/NULLIF(income-expenses, 0)  AS P_E_ratio
        FROM investor data;

Si dado el caso son iguales, *price_per_share* se dividira por NULL, y no por cero.

### GREATEST

Aqui retornara 14:

    SELECT GREATEST(4,14,-1,11);

### LEAST

Y aqui -1:

    SELECT LEAST(4,14,-1,11);

### BLOQUES ANONIMOS

Agregar una nueva columna donde indique si el pasajero es mayor de edad o no:

    SELECT id, name, home_address, birthdate,
    CASE
	WHEN birthdate > '2005-09-01' THEN
	'menor'
	ELSE
	'mayor'
	END
    	FROM public.pasajero;

![](https://i.imgur.com/gGRtR4Z.png)

#### RETO:

Consultar la informacion de los pasajeros agregando una columna adicional, quienes empiezan su nombre por la letra *p*, y ademas quienes son mayores de edad o no.

#### SOL

    SELECT name, birthdate,
		CASE
		WHEN birthdate >= '1996-08-04' THEN
		'menor'
		ELSE
		'mayor'
		END
    FROM public.pasajero
    WHERE name ILIKE 'p%';

![](https://i.imgur.com/XDejMFE.png)

#### Funcion especial - date_part

This function allows you to retrieve subfields e.g., year, month, week from a date or time value.

    SELECT date_part('year',TIMESTAMP '2017-01-01');

Retornaria el año 2017    

Otro ejemplo:

    SELECT date_part('year', birthdate) as edad
    FROM pasajero;

![](https://i.imgur.com/ZE4O9Es.png)


#### Ejemplo

Seleccionar los pasajeros que sean mayores de 50 años para que tengan un descuento:

    SELECT name AS nombre
    FROM pasajero
    WHERE date_part('year', current_date) - date_part('year', birthdate) >= 50;

21 coincidencias.

## Vistas

La consulta la vamos a repetir muchas veces, hay dos tipos:

- volatil: siempre se consulta a la BD
- materializada: se realiza una vez, quea guardada en memoria, y futuras busquedas buscan en la memoria en lugar de la BD.

### Volatil

Supongamos que queremos convertir la script anterior en una consulta volatil:

Click derecho sobre *views* 

![](https://i.imgur.com/alLeZs7.png)

Le damos un nombre cualquiera (rango_view), y copiamos el codigo en *CODE*, y guardamos.

Y probemos:

    SELECT * FROM rango_view; 👈

### Vistas Materializadas

Click derecho sobre *materialized views*:

![](https://i.imgur.com/2TGRmYI.png)

¿Que tal si creamos una vista materializada para los inicios despues de las 9PM?

![](https://i.imgur.com/5kXnxQl.png)

Le damos un nombre cualquiera(despues_noche_mview), copiamos el codigo en *definition*:

    SELECT * 
    FROM viaje
    WHERE date_part('hour', inicio) >= 21;


En la pestaña de *storage*, hay la opcion de crearlo *With_data* es decir se crea la consulta, y en ese momento queda con datos, o de lo contrario queda vacia. Le damos la opcion de que no.

![](https://i.imgur.com/r5BXnuC.png)

La primera consulta, estara vacia:

    SELECT * FROM despues_noche_mview;

'materialized view "despues_noche_mview" has not been populated', asi que:

    REFRESH MATERIALIZED VIEW despues_noche_mview;

![](https://i.imgur.com/0YjgDeE.png)

Borremos ahora una fila, la que tiene el ID 4.

Si volvemos a consultar la vista materializada, sin habeer refrescado nos seguira apareciendo la fila con el id=4. Nuevamente, y volvemos a consultar:

    REFRESH MATERIALIZED VIEW despues_noche_mview;
    SELECT * FROM despues_noche_mview;

## PL/PgSQL

Stands for Procedural Languaje/PosgreSQL Structured Querry Language. Tambien conocidos en Español como *Procedimientos Almacenados*, hacen parte del motor de Postgres, y es tal vez una de las razones por las cuales los desarrolladores empiezan a usar este motor. Ayuda a desarrollar codigo directamente sobre la DB.

Tener en cuenta que las PL se usan en su gran mayoria para consultas, una PL que no tenga que ver con consultas en realidad no sabemos que tan util puede llegar a ser.

### Bloque de codigo

Se ejecuta con la palabre **DO**, y va encerrada en simbolo doble de pesos. 

![](https://i.imgur.com/D5pLago.png) 

### Añadiendo Variables

Con el Statement *DECLARE*. Tener en cuenta que las PL se usan en su gran mayoria para consultas, una PL que no tenga que ver con consultas en realidad no sabemos que tan util puede llegar a ser.

El tipo de datos que nos sirve para almacernar informacion de una fila es *record*, y el nombre de la variable *rec* viene a la izquierda. Para asignare un valor a una variable, hacerlo con **:=**.

Para el ejemplo, iniciemolo en NULL, aunque no es necesario. 

![](https://i.imgur.com/1YMKBLs.png)

https://www.postgresql.org/docs/9.2/plpgsql.html


- RAISE NOTICE muestra informacion en *Messages*, o blog de notificaciones

- *FOR rec IN SELECT * FROM pasajero LIMIT 10 LOOP*. Cada una de las filas de la consulta quedara almacenada en la variable *rec*. Y para acceder a todos sus atributos usamos punto, tal cosa. 

CODIGO:

    DO $$
        DECLARE
        rec record := NULL;	
    BEGIN
        FOR rec IN SELECT * FROM pasajero LIMIT 10 LOOP
            RAISE NOTICE 'un pasajero se llama %', rec.name;
        END LOOP;
    END
    $$
    

### Hacer modificaciones de variables dentro de las funciones

Por ejemplo un contador.y lo inicializamos con un valor.

![](https://i.imgur.com/irYezdk.png)

### Encapsulamiento

![](https://i.imgur.com/62bqqAp.png)

- Para meter el bloque de codigo anterior en una funcion. Remplazamos el DO con el CREATE FUNCTION. 

- Debe retorna algo, para este caso, no retorna nada. 
- Usar el statement *AS*
- Es indispensable indicarle que tipo de lenguage usa, el bloque de codigo es PLPGSQL, pero Postgres soporta, usando algunas librerias, ejecutar codigo de Python, C++, o SQL basico. Si fuera este el caso colocariamos simplemente LANGUAGE SQL. 

Y en el arbol de la izquierda aparecera una nueva funcion:

![](https://i.imgur.com/UE6mS0W.png)

Y para llamarla, con un SELECT:

![](https://i.imgur.com/OdKliy7.png)

Como ves no retorna nada, porque le indicamos asi lo hiciera, pero en mensajes veras el resultado

![](https://i.imgur.com/OdKliy7.png)

![](https://i.imgur.com/CKAs8fE.png)




#### Codigo Fuente(Simplemente copialo)

CREATE FUNCTION imporantePL() 
 RETURNS void
AS $$
 DECLARE
 rec record := NULL;
 contador integer := 0;
 BEGIN
  FOR rec IN SELECT * FROM pasajero LIMIT 10 LOOP
  	RAISE NOTICE 'un pasajero se llama %', rec.name;
	contador := contador + 1;
  END LOOP;
  RAISE NOTICE 'Numero de Filas Consultadas %', contador;
 END
$$
LANGUAGE PLPGSQL;

### Modificando la funcion para retornar un valor

Si realizamos algun cambio, por ejemplo para que retorne el valor de contador, saldra un error, porque la funcion ya existe, asi que la podemos borrar o agregar *CREATE OR REPLACE*. Sin embargo seguira persistiendo un error, de que no es posible cambiar el tipo de dato que retorna una funcion asi que lo mejor es borrarla:

    DROP FUNCTION imporantepl()

Y a continuacion volver a crear la funcion:

![](https://i.imgur.com/cHq4hF0.png)

Script completo: 👇

--DROP FUNCTION importantePL();

CREATE OR REPLACE FUNCTION importantePL() 
 RETURNS integer
AS $$
 DECLARE
 rec record := NULL;
 contador integer := 0;
 BEGIN
  FOR rec IN SELECT * FROM pasajero LIMIT 10 LOOP
  	RAISE NOTICE 'un pasajero se llama %', rec.name;
	contador := contador + 1;
  END LOOP;
  RAISE NOTICE 'Numero de Filas Consultadas %', contador;
  RETURN contador;
 END
$$
LANGUAGE PLPGSQL;

-- SELECT importantePL();

### Creando PL usando pgadmin

Sobre funciones le damos crear:

![](https://i.imgur.com/UE6mS0W.png)

- Le damos un nombre a la funcion (importantePL1)
- En la pestaña de Definition, nos pide lo que va retornar, si no queremos que retorne nada, escribir *void*. Para este caso *integer*. Tambien seleccionar el tipo de lenguaje: PLPGSQL

- En codigo, copiamos el bloque de codigo, preferiblemente sin las llaves


        DECLARE
        rec record := NULL;
        contador integer := 0;
        BEGIN
        FOR rec IN SELECT * FROM pasajero LIMIT 10 LOOP
            RAISE NOTICE 'un pasajero se llama %', rec.name;
            contador := contador + 1;
        END LOOP;
        RAISE NOTICE 'Numero de Filas Consultadas %', contador;
        RETURN contador;
        END


- En options hay cosas interesantes, que por ahora no manipularemos. 

Le damos crear. Y la ejecutamos

    SELECT importantePL1();

y no funcionara 😣... asi que click derecho sobre la funcion, y que nos cree el Script de Seleccion por defecto:

    SELECT public."importantePL1"()

Y ahora si!

Para evitar este problemilla, nombra las funciones todo en minusculas

## Triggers

Tambien conocidos como disparadores, que se "disparan" cuando hay un:

- insert
- update
- delete

https://platzi.com/clases/4203-mysql-mariadb/54614-triggers-compuestos/

### Modificar la funcion anterior: importantepl1

Para que siga retornando el numero de filas, e imprima solo el nombre del ultimo pasajero:

![](https://i.imgur.com/OlJazpd.png)

CODIGO

DECLARE
 rec record := NULL;
 contador integer := 0;
 BEGIN
  SELECT count(*) INTO contador FROM pasajero;	
  SELECT * INTO rec FROM pasajero WHERE id=contador;
  
  RAISE NOTICE 'Nombre ultimo pasajero %', rec.name;
  RAISE NOTICE 'Numero de Filas: %', contador;
  
  RETURN contador;
 END

### Preparando el terreno 

Haremos que la funcion creada en el modulo anterior se ejecute justo despues de un INSERT en la tabla de pasajeros. Y guardar el valor de contador en una tabla adicional(conteopasajeros), y cuando ocurre.

Creamos la nueva tabla:

![](https://i.imgur.com/AGRotl2.png)

#### Volviendo a modificar la funcion anterior

Para que cada vez que se ejecute haga un INSERT en la tabla *conteopasajeros*:

![](https://i.imgur.com/bgLVYMT.png)


CODIGO

DECLARE
 rec record := NULL;
 contador integer := 0;
 BEGIN
  SELECT count(*) INTO contador FROM pasajero;	
  SELECT * INTO rec FROM pasajero WHERE id=contador;
  
  RAISE NOTICE 'Nombre ultimo pasajero %', rec.name;
  RAISE NOTICE 'Numero de Filas: %', contador;
  
  INSERT INTO conteopasajeros (contador, tiempo)
  VALUES (contador, now());
  RETURN contador;
 END

#### Comprobando

    SELECT importantepl1()

Dando como resultado la insercion en la tabla *conteopasajeros*

![](https://i.imgur.com/jPzdrsO.png)

#### Creando una funcion nueva que retorne trigger

Para usar la funcion en un trigger, la funcion debe retornas un trigger, por tanto, y para no tener que empezar de cero, usamos un truquito para copiar toda la funcion asi sin mas:

- Sobre la funcion: Scripts-->Create Script

![](https://i.imgur.com/8qBesTx.png)

Modificamos lo que haya que modificar:

- Fijate ya no retorna nada al final de la funcion.

![](https://i.imgur.com/eSVfffF.png)

Asegurate de borrar primero la funcion:

    DROP FUNCTION IF EXISTS public.importantepl1();

Y ahora si vuelvela a crear con el codigo mostrado ✅

La funcion ya no estara sobre el arbol de Functions, sino en el de *Trigger Functions*

![](https://i.imgur.com/C3gU3Eo.png)





CODIGO

CREATE OR REPLACE FUNCTION public.importantepl1(
	)
    RETURNS TRIGGER
    LANGUAGE 'plpgsql'
    COST 100
    VOLATILE PARALLEL UNSAFE
AS $BODY$
DECLARE
 rec record := NULL;
 contador integer := 0;
 BEGIN
  SELECT count(*) INTO contador FROM pasajero;	
  SELECT * INTO rec FROM pasajero WHERE id=contador;
  
  RAISE NOTICE 'Nombre ultimo pasajero %', rec.name;
  RAISE NOTICE 'Numero de Filas: %', contador;
  
  INSERT INTO conteopasajeros (contador, tiempo)
  VALUES (contador, now());
 END
$BODY$;

ALTER FUNCTION public.importantepl1()
    OWNER TO curso;


### Creando el Trigger

Los triggers se disparan cuando hay un INSERT, un UPDATE o un DELETE, para nuestro case es cuando hay un INSERT.

- Se puede ejecutar Antes o Despues o INSTEAD: AFTER or BEFORE or INSTEAD OF. Si queremos que se ejecute antes del INSERT, usamos BEFORE

- FOR EACH ROW, quiere decir por cada fila que se este insertando para este caso

![](https://i.imgur.com/bHvCRPp.png)

Una vez ejecutemos el Sript, debio haber quedado creada la conexion entre la PL, el Trigger y la Tabla. Observar creo el trigger sobre la tabla pasajeros

![](https://i.imgur.com/HWxgTLZ.png)

### Insertando una fila en la tabla Pasajeros

Probamos y falla:

![](https://i.imgur.com/aPm4xDj.png)


Dice que nuestro trigger se ejcuto sin un RETURN. En este caso se le debe confirmar al motor de la base de datos si el cambio se hace o no.

Para empezar, los triggers tienen dos variables globales muy importanes *NEW* y *OLD*. OLD es lo que estaba antes del cambio, y NEW es el cambio. 

OLD y NEW tiene mas sentido en un UPDATE, donde tendriamso la informacion antes del UPDATE(OLD), y despues del UPDATE(NEW).

Para este caso nuestro script de INSERT, OLD no tiene sentido, mientras que NEW tiene toda la info. Usamos NEW.

👉 COn es statement *RETURN NEW*, indicamos que lo que estamos aceptando es lo que estamos insertando.

#### Corrigiendo el error

Volviendo a borrar el Trigger, porque cambia el tipo de retorno:

- Borrar el trigger *mitrigger*: sobre el arbol si asi lo deseas. 
- Borrar la funcion *importantepl1*

    DROP FUNCTION IF EXISTS public.importantepl1();

- Corrigiendo la funcion *importantepl1*:

![](https://i.imgur.com/L2kPkvD.png)

- volviendo a crear el trigger;

![](https://i.imgur.com/bHvCRPp.png)

- Volviendo a probar el INSERT, y como vez hay una nueva fila 😁

![](https://i.imgur.com/2lDRodR.png)

CODIGO CORREGIDO Y DEFINITIVO

CREATE OR REPLACE FUNCTION public.importantepl1()
    RETURNS trigger
    LANGUAGE 'plpgsql'
    COST 100
    VOLATILE NOT LEAKPROOF
AS $BODY$
DECLARE
 rec record := NULL;
 contador integer := 0;
 BEGIN
  SELECT count(*) INTO contador FROM pasajero;	
  SELECT * INTO rec FROM pasajero WHERE id=contador;
  
  RAISE NOTICE 'Nombre ultimo pasajero %', rec.name;
  RAISE NOTICE 'Numero de Filas: %', contador;
  
  INSERT INTO conteopasajeros (contador, tiempo)
  VALUES (contador, now());
  RETURN NEW;
  
 END
$BODY$;

ALTER FUNCTION public.importantepl1()
    OWNER TO curso;


#### Ejercicio

Modificar o crear un nuevo trigger para que se ejecute cuando se borre un registro de la tabla pasajeros:

- Creamos la funcion *importantepl2*, es una copia casi identida de importantepl1
- Creamos la el trigger *mitrigger2*

![](https://i.imgur.com/qCYPf4Q.png)


##### Testiando

Despues de hacer un JOIN para identificar todos los viajes del pasajero 101, y borrar dichos viajes:

    SELECT * FROM PASAJERO
    JOIN viaje 
    ON (viaje.id_pasajero = pasajero.id)
    WHERE id_pasajero = 101;

Se borra el pasajero 101:

    DELETE FROM public.pasajero
	WHERE id=101;

Vemos que se activo el trigger:

![](https://i.imgur.com/iX8xyla.png)

Verificando en la tabla *conteopasajeros*:

![](https://i.imgur.com/iz6kR6L.png)

